In [1]:
import random
import torch.nn as nn
import torch
import pickle
import pandas as pd
from pandas import Series, DataFrame
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False)
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, matthews_corrcoef, f1_score, precision_score, recall_score
import numpy as np
import torch.optim as optim
folder = "/data/AIpep-clean/"
import matplotlib.pyplot as plt
from vocabulary import Vocabulary
from datasethem import Dataset
from datasethem import collate_fn_no_activity as collate_fn
from models import Generator
from tqdm.autonotebook  import trange, tqdm
import os
from collections import defaultdict

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# Load data

In [2]:
df = pd.read_pickle(folder + "pickles/DAASP_RNN_dataset_with_hemolysis.plk")

df_training = df.query("Set == 'training' and aureus == True and isNotHemolytic==1")
df_test = df.query("Set == 'test' and aureus == True and isNotHemolytic==1")

if torch.cuda.is_available():
    device = "cuda" 
else:
    device = "cpu" 

In [3]:
print("Against S. aureus:\nactive training "+ str(len(df_training[df_training["activity"]==1])) \
      + "\nactive test " + str(len(df_test[df_test["activity"]==1])) \
      + "\ninactive training "+ str(len(df_training[df_training["activity"]==0])) \
      + "\ninactive test " + str(len(df_test[df_test["activity"]==0])))

Against S. aureus:
active training 321
active test 89
inactive training 137
inactive test 39


# Define helper functions

In [4]:
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return category_i

def nan_equal(a,b):
    try:
        np.testing.assert_equal(a,b)
    except AssertionError:
        return False
    return True

def models_are_equal(model1, model2):
    model1.vocabulary == model2.vocabulary
    model1.hidden_size == model2.hidden_size
    for a,b in zip(model1.model.parameters(), model2.model.parameters()):
        if nan_equal(a.detach().numpy(), b.detach().numpy()) == True:
            print("true")

# Define hyper parameters

In [5]:
n_embedding  = 100
n_hidden = 400
n_layers = 2
n_epoch = 200
learning_rate = 0.00001
momentum = 0.9
batch_size = 10
epoch = 22

# Loading and Training

In [6]:
if not os.path.exists(folder+"pickles/generator_TL_grampos_results_hem.pkl"):
    model = Generator.load_from_file(folder+"models/RNN-generator/ep{}.pkl".format(epoch))
    model.to(device)
    vocabulary = model.vocabulary
    
    df_training_active = df_training.query("activity == 1")
    df_test_active = df_test.query("activity == 1")
    df_training_inactive = df_training.query("activity == 0")
    df_test_inactive = df_test.query("activity == 0")

    training_dataset_active = Dataset(df_training_active, vocabulary, with_activity=False)
    test_dataset_active = Dataset(df_test_active, vocabulary, with_activity=False)
    training_dataset_inactive = Dataset(df_training_inactive, vocabulary, with_activity=False)
    test_dataset_inactive = Dataset(df_test_inactive, vocabulary, with_activity=False)

    optimizer = optim.SGD(model.model.parameters(), lr = learning_rate, momentum=momentum)

    # the only one used for training
    training_dataloader_active = torch.utils.data.DataLoader(training_dataset_active, batch_size=batch_size, shuffle=True, collate_fn = collate_fn, drop_last=True, pin_memory=True, num_workers=4)

    # used for evaluation
    test_dataloader_active = torch.utils.data.DataLoader(test_dataset_active, batch_size=batch_size, shuffle=False, collate_fn = collate_fn, drop_last=False, pin_memory=True, num_workers=4)
    training_dataloader_inactive = torch.utils.data.DataLoader(training_dataset_inactive, batch_size=batch_size, shuffle=False, collate_fn = collate_fn, drop_last=False, pin_memory=True, num_workers=4)
    test_dataloader_inactive = torch.utils.data.DataLoader(test_dataset_inactive, batch_size=batch_size, shuffle=False, collate_fn = collate_fn, drop_last=False, pin_memory=True, num_workers=4)
    training_dataloader_active_eval = torch.utils.data.DataLoader(training_dataset_active, batch_size=batch_size, shuffle=False, collate_fn = collate_fn, drop_last=False, pin_memory=True, num_workers=4)

    training_dictionary = {}

    for e in trange(1, n_epoch + 1):
        print("Epoch {}".format(e))
        for i_batch, sample_batched in tqdm(enumerate(training_dataloader_active), total=len(training_dataloader_active) ):

            seq_batched = sample_batched[0].to(model.device, non_blocking=True) 
            seq_lengths = sample_batched[1].to(model.device, non_blocking=True)

            nll = model.likelihood(seq_batched, seq_lengths)

            loss = nll.mean()

            optimizer.zero_grad()
            loss.backward()  
            torch.nn.utils.clip_grad_value_(model.model.parameters(), 2)
            optimizer.step()

        model.save(folder+"models/RNN-generator-TL-hem/grampos_ep{}.pkl".format(e))


        print("\tExample Sequences")
        sampled_seq = model.sample(5)
        for s in sampled_seq:
            print("\t\t{}".format(model.vocabulary.tensor_to_seq(s, debug=True)))

        nll_training = []
        with torch.no_grad():
            for i_batch, sample_batched in enumerate(training_dataloader_active_eval):    
                seq_batched = sample_batched[0].to(model.device, non_blocking=True) 
                seq_lengths = sample_batched[1].to(model.device, non_blocking=True) 

                nll_training += model.likelihood(seq_batched, seq_lengths)

        nll_training_active_mean = torch.stack(nll_training).mean().item()
        print("\tNLL Train Active: {}".format(nll_training_active_mean))
        del nll_training

        nll_test = []
        with torch.no_grad():
            for i_batch, sample_batched in enumerate(test_dataloader_active):    
                seq_batched = sample_batched[0].to(model.device, non_blocking=True) 
                seq_lengths = sample_batched[1].to(model.device, non_blocking=True) 

                nll_test += model.likelihood(seq_batched, seq_lengths)

        nll_test_active_mean = torch.stack(nll_test).mean().item()
        print("\tNLL Test Active: {}".format(nll_test_active_mean))
        del nll_test

        nll_training = []
        with torch.no_grad():
            for i_batch, sample_batched in enumerate(training_dataloader_inactive):    
                seq_batched = sample_batched[0].to(model.device, non_blocking=True) 
                seq_lengths = sample_batched[1].to(model.device, non_blocking=True) 

                nll_training += model.likelihood(seq_batched, seq_lengths)

        nll_training_inactive_mean = torch.stack(nll_training).mean().item()
        print("\tNLL Train Inactive: {}".format(nll_training_inactive_mean))
        del nll_training

        nll_test = []
        with torch.no_grad():
            for i_batch, sample_batched in enumerate(test_dataloader_inactive):    
                seq_batched = sample_batched[0].to(model.device, non_blocking=True) 
                seq_lengths = sample_batched[1].to(model.device, non_blocking=True) 

                nll_test += model.likelihood(seq_batched, seq_lengths)

        nll_test_inactive_mean = torch.stack(nll_test).mean().item()
        print("\tNLL Test Inactive: {}".format(nll_test_inactive_mean))
        del nll_test
        print()

        training_dictionary[e]=[nll_training_active_mean, nll_test_active_mean, nll_training_inactive_mean, nll_test_inactive_mean]
    
    with open(folder+"pickles/generator_TL_grampos_results_hem.pkl",'wb') as fd:
        pickle.dump(training_dictionary, fd)
    
else:
    with open(folder+"pickles/generator_TL_grampos_results_hem.pkl",'rb') as fd:
        training_dictionary = pickle.load(fd)

min_nll_test_active = float("inf")
for epoch, training_values in training_dictionary.items():
    nll_test_active = training_values[1]

    if nll_test_active < min_nll_test_active:
        best_epoch = epoch
        min_nll_test_active = nll_test_active

Epoch 1



	Example Sequences
		<start>RGDKRPYRPRPRPPRRIYNRG<end>
		<start>RAKRHHGYKRKFH<end><pad><pad><pad><pad><pad><pad><pad><pad>
		<start>ILPWKLPWWPWRR<end><pad><pad><pad><pad><pad><pad><pad><pad>
		<start>GLLSLLGKALLHLGQLL<end><pad><pad><pad><pad>
		<start>vkvkaavkalhlGkkahy<end><pad><pad><pad>
	NLL Train Active: 17.657602310180664
	NLL Test Active: 25.938371658325195
	NLL Train Inactive: 35.9663200378418
	NLL Test Inactive: 39.14516067504883

Epoch 2



	Example Sequences
		<start>VnKVPLKKLRQIGRKRISKIRPHRTK<end>
		<start>LRCARRHRHRH<end><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
		<start>KWWLfKpWKpGWK<end><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
		<start>WFWHIKRVVRGWRRWV<end><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
		<start>APKKVKKLAKRGFKKYKNYK<end><pad><pad><pad><pad><pad><pad>
	NLL Train Active: 17.420150756835938
	NLL Test Active: 25.76340103149414
	NLL Train Inactive: 35.85044860839844
	NLL Test Inactive: 39.069820404052734

Epoch 3



	Example Sequences
		<start>LLkKVFKRLaKGI<end><pad><pad><pad><pad>
		<start>FFHHIFRAIVHVVKTVG<end>
		<start>IRIIWRLCIRVWR<end><pad><pad><pad><pad>
		<start>ILPWKWPWKWPWKR<end><pad><pad><pad>
		<start>WWRWRWRpGRWWWRR<end><pad><pad>
	NLL Train Active: 17.22064208984375
	NLL Test Active: 25.624237060546875
	NLL Train Inactive: 35.75004959106445
	NLL Test Inactive: 38.993812561035156

Epoch 4



	Example Sequences
		<start>FFGWLIKGAIHAPKHIHPK<end><pad><pad><pad><pad><pad><pad><pad>
		<start>FLGKiLRLGKGLKGLL<end><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
		<start>VDKKPYKPRRRPPRRIYNR<end><pad><pad><pad><pad><pad><pad><pad>
		<start>KWSKIWPGVKKLIKGLF<end><pad><pad><pad><pad><pad><pad><pad><pad><pad>
		<start>GFWGKVADVIGRVAGRVLGGAVTRRS<end>
	NLL Train Active: 17.04739761352539
	NLL Test Active: 25.5029354095459
	NLL Train Inactive: 35.6693229675293
	NLL Test Inactive: 38.937294006347656

Epoch 5


...


	Example Sequences
		<start>RRLRKWRPKLKKLLKKL<end>
		<start>KWKKLLKKPLLK<end><pad><pad><pad><pad><pad>
		<start>GGHGAGKILK<end><pad><pad><pad><pad><pad><pad><pad>
		<start>GRLRKWRKKY<end><pad><pad><pad><pad><pad><pad><pad>
		<start>FRRFFFFFRRFRRF<end><pad><pad><pad>
	NLL Train Active: 11.425165176391602
	NLL Test Active: 24.093870162963867
	NLL Train Inactive: 35.22422409057617
	NLL Test Inactive: 38.23530197143555

Epoch 197



	Example Sequences
		<start>RWKWWRRVL<end><pad><pad><pad><pad><pad><pad>
		<start>KRFRPLVKKLAKKIL<end>
		<start>LLKWLKKLLKK<end><pad><pad><pad><pad>
		<start>RWRWRWWR<end><pad><pad><pad><pad><pad><pad><pad>
		<start>KLWKLWKKWLKK<end><pad><pad><pad>
	NLL Train Active: 11.413411140441895
	NLL Test Active: 24.09268569946289
	NLL Train Inactive: 35.22749328613281
	NLL Test Inactive: 38.23994827270508

Epoch 198



	Example Sequences
		<start>ILGAVLKVATTGLPALISWIKRKRQQ<end>
		<start>GIIKPLASKWPKWKPWRR<end><pad><pad><pad><pad><pad><pad><pad><pad>
		<start>KIGAKLAKTAVKLLK<end><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
		<start>RWRWWIRWRWRW<end><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
		<start>FLGWLFKVASK<end><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
	NLL Train Active: 11.401329040527344
	NLL Test Active: 24.093080520629883
	NLL Train Inactive: 35.23316955566406
	NLL Test Inactive: 38.24077606201172

Epoch 199



	Example Sequences
		<start>KILGPVIKKLLKGLF<end><pad><pad><pad><pad><pad><pad><pad><pad><pad>
		<start>GLFSKVGNVGRNFLGAVGGKIMKG<end>
		<start>SRWRWRPFFSLSPRTIAMRKFL<end><pad><pad>
		<start>FLGVLFKALSKLL<end><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
		<start>LLLIVVLKKV<end><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
	NLL Train Active: 11.38957691192627
	NLL Test Active: 24.09497833251953
	NLL Train Inactive: 35.23904037475586
	NLL Test Inactive: 38.247276306152344

Epoch 200



	Example Sequences
		<start>KILGKWKPWWWRR<end><pad><pad><pad><pad><pad>
		<start>GRFRRLRKKTRKRLKKIG<end>
		<start>SKKLFKKIPKFLHLAKKF<end>
		<start>AARIILRLRI<end><pad><pad><pad><pad><pad><pad><pad><pad>
		<start>FLGALFKALSKLL<end><pad><pad><pad><pad><pad>
	NLL Train Active: 11.377496719360352
	NLL Test Active: 24.097930908203125
	NLL Train Inactive: 35.24226379394531
	NLL Test Inactive: 38.246925354003906




# Sampling evaluation

In [7]:
print(best_epoch)
model = Generator.load_from_file(folder+"models/RNN-generator-TL-hem/grampos_ep{}.pkl".format(best_epoch))

149


142

In [8]:
training_seq = df_training.Sequence.values.tolist()

def _sample(model, n):
    sampled_seq = model.sample(n)
    sequences = []
    for s in sampled_seq:
        sequences.append(model.vocabulary.tensor_to_seq(s))
    return sequences

def novelty(seqs, list_):
    novel_seq = []
    for s in seqs:
        if s not in list_:
            novel_seq.append(s)
    return novel_seq, (len(novel_seq)/len(seqs))*100

def is_in_training(seq, list_ = training_seq):
    if seq not in list_:
        return False
    else:
        return True

def uniqueness(seqs):
    unique_seqs = defaultdict(int)
    for s in seqs:
        unique_seqs[s] += 1
    return unique_seqs, (len(unique_seqs)/len(seqs))*100

In [9]:
# sample
seqs = _sample(model, 50000)
unique_seqs, perc_uniqueness = uniqueness(seqs)
notintraining_seqs, perc_novelty = novelty(unique_seqs, training_seq)


# create dataframe
df_generated = pd.DataFrame(list(unique_seqs.keys()), columns =['Sequence']) 
df_generated["Repetition"] = df_generated["Sequence"].map(lambda x: unique_seqs[x])
df_generated["inTraining"] = df_generated["Sequence"].map(is_in_training)
df_generated["Set"] = "generated-TL-GP-hem"

# save
df_generated.to_pickle(folder+"pickles/Generated-TL-grampos-hem.pkl")

In [10]:
print(perc_uniqueness, perc_novelty)

82.284 99.48957269943124


82.284 99.48957269943124